# Quantifying delays between exposures.
Craig Lage - 07-Jan-26

In [ ]:
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from astropy.time import Time, TimeDelta
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors, plotMountErrors, \
        getAltAzOverPeriod, getAzElRotHexDataForExposure
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId
from lsst.summit.extras.slewTimingSimonyi import inPositionTopics, getDomeData
import warnings
warnings.filterwarnings("ignore")

In [ ]:
client = makeEfdClient()
butler = butlerUtils.makeDefaultButler("LSSTCam", embargo=True)

In [ ]:
def getKeyEvents(expId, printLastInPosition=False):
    events = {}
    deltas = {}
    dataId1 = {'exposure':expId, 'instrument':'LSSTCam'}
    expRecord1 = getExpRecordFromDataId(butler, dataId1)
    dataId2 = {'exposure':expId + 1, 'instrument':'LSSTCam'}
    expRecord2 = getExpRecordFromDataId(butler, dataId2)
    deltas['Azimuth'] = abs(expRecord1.azimuth - expRecord2.azimuth)
    deltas['Elevation'] = abs(expRecord1.zenith_angle - expRecord2.zenith_angle)
    shutterClose = expRecord1.timespan.end.utc
    events['shutterClose'] = shutterClose
    t0 = events['shutterClose'].unix_tai
    shutterOpen = expRecord2.timespan.begin.utc
    slew = getEfdData(client, 'lsst.sal.MTPtg.command_raDecTarget', begin=shutterClose, end=shutterOpen)
    if len(slew) == 0:
        events['shutterOpen'] = shutterOpen
        eventsToPlot = ['shutterClose', 'shutterOpen']
        times = []
        for event in eventsToPlot:
            times.append(events[event].unix_tai - t0)
        intervals = []
        for i, time in enumerate(times):
            if i == len(times) - 1:
                break
            intervals.append((time, times[i + 1] - time))
        return [deltas, events, eventsToPlot, None, times, intervals]
    startSlew = Time(slew.index[0])
    events['startSlew'] = startSlew
    inPositionTimes = {}
    lastInPositionTime = shutterClose.unix_tai
    lastInPosition = 'Elevation'
    for key in inPositionTopics.keys():
        if key == 'Hexapod':
            continue
        if key == 'Dome':
            domeData, domeBelowThreshold = getDomeData(client, begin=shutterClose, end=shutterOpen,
                                           prePadding=0.0, postPadding=0.0)
            if len(domeBelowThreshold) > 0:
                inPositionTime = Time(domeBelowThreshold.index[-1])
                inPositionTimes[key] = inPositionTime
        else:
            topic = inPositionTopics[key]
            inPosition = getEfdData(client, topic, begin=shutterClose, end=shutterOpen)
            if len(inPosition) > 0:
                inPositionTime = Time(inPosition.index[-1])
                inPositionTimes[key] = inPositionTime
        if inPositionTime.unix_tai > lastInPositionTime:
            lastInPositionTime = inPositionTime.unix_tai
            lastInPosition = key
    events['inPositionTimes'] = inPositionTimes
    lastInPositionTime = Time(lastInPositionTime, format='unix_tai', scale='utc')
    if printLastInPosition:
        print(lastInPosition)
    events['inPosition'] = lastInPositionTime
    slewFlag = getEfdData(client, 'lsst.sal.MTM1M3.command_clearSlewFlag', begin=shutterClose, end=shutterOpen)
    events['clearSlewFlag'] = Time(slewFlag.index[-1])
    events['shutterOpen'] = shutterOpen
    eventsToPlot = list(events.keys())
    if 'inPositionTimes' in eventsToPlot:
        eventsToPlot.remove('inPositionTimes')
    times = []
    for event in eventsToPlot:
        times.append(events[event].unix_tai - t0)
    intervals = []
    for i, time in enumerate(times):
        if i == len(times) - 1:
            break
        intervals.append((time, times[i + 1] - time))

    return [deltas, events, eventsToPlot, lastInPosition, times, intervals]
    
    

In [ ]:
def plotEvents(ax, events, eventsToPlot, times, intervals):
    colors = ['blue', 'red', 'coral', 'green']
    t0 = events['shutterClose'].unix_tai
    ax.broken_barh(intervals, (0, 1), facecolors=colors, alpha=0.3)
    
    # Labels
    ax.set_xlim(times[0], times[-1])
    ax.set_ylim(0, 1)
    ax.set_yticks([])
    ax.set_xticks(times)
    ax.set_xticklabels(eventsToPlot, rotation=90)
    if 'inPositionTimes' in list(events.keys()):
        for key in events['inPositionTimes'].keys():
            inPos = events['inPositionTimes'][key].unix_tai
            ax.axvline(inPos - t0, ls='--', color='black')
            ax.text(inPos - t0, 0.1, key, rotation=90)
    ax1 = ax.secondary_xaxis(location='top')
    tlabels = [f'{t:.1f}' for t in times]
    ax1.set_xticks(times)
    ax1.set_xticklabels(tlabels)
    ax1.set_xlabel("Time(seconds)")
    return

    


In [ ]:
for expId in [2026010900144]:#[2026010300685, 2026010300706, 2026010300683]:
    [deltas, events, eventsToPlot, lastInPosition, times, intervals] = getKeyEvents(expId, printLastInPosition=True)
    fig, ax = plt.subplots(figsize=(12, 2))
    plotEvents(ax, events, eventsToPlot, times, intervals)
    fig.savefig(f"/home/c/cslage/u/MTMount/between_image_timing/Timing_{expId}.png",
                bbox_inches='tight', pad_inches=1.2)

In [ ]:
data = {}
for dayObs in [20260113]:#[20251223, 20251224, 20260101, 20260102, 
#               20260103, 20260104]:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    print(dayObs, len(exposureList))
    for [id,record] in exposureList:
        if record.observation_type not in ['science']:
            continue
        try:
            expId = record.id
            thisData = getKeyEvents(expId)
            data[expId] = thisData
            #print(f"{expId} succeeded!")
        except:
            #print(f"{expId} failed!")
            continue
    print(f"Finished {dayObs}")
filename = f"/home/c/cslage/u/MTMount/between_image_timing/timing_results_{dayObs}.pkl"
with open(filename, 'wb') as f:
    pkl.dump(data, f)


In [ ]:
filename = f"/home/c/cslage/u/MTMount/between_image_timing/timing_results_08Jan26.pkl"
with open(filename, 'rb') as f:
    data = pkl.load(f)
len(list(data.keys()))

In [ ]:
total_times = []
interval_times = [[], [], [], []]
lastInPositions = []
maxDeltas = []
for key in data.keys():
    [deltas, events, eventsToPlot, lastInPosition, times, intervals] = data[key]
    total_times.append(times[-1])
    if len(intervals) == 4:
        for i in range(4):
            interval_times[i].append(intervals[i][1])
        maxDeltas.append(max(deltas['Azimuth'], deltas['Elevation']))
    if lastInPosition:
        lastInPositions.append(lastInPosition)
    
        
    

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
plt.suptitle(f"Total time of FBS intervals - {dayObs}", fontsize=18)
ax.hist(total_times, bins=50, range=(0,100))
ax.set_yscale('log')
ax.set_xlabel("Duration(seconds)")

fig.savefig(f"/home/c/cslage/u/MTMount/between_image_timing/Total_Times_{dayObs}.png",
                bbox_inches='tight', pad_inches=1.2)

In [ ]:
names = ['Shutter close to Slew start', 'Slew start to allInPosition',
         'allInPosition to clearSlewFlag', 'clearSlewFlag to shutterOpen']
ranges = [10, 200, 10, 5]
fig, axs = plt.subplots(2,2,figsize=(10,10))
plt.suptitle(f"Time breakdown of FBS intervals - {dayObs}", fontsize=18)
axes = [axs[0][0], axs[0][1], axs[1][0], axs[1][1]]
for i in range(4):
    axes[i].hist(interval_times[i], bins=50, range=(0,ranges[i]))
    axes[i].set_title(names[i])
    #if i == 1:
    axes[i].set_yscale('log')
    axes[i].set_xlabel("Duration(seconds)")

fig.savefig(f"/home/c/cslage/u/MTMount/between_image_timing/Detailed_Timing_{dayObs}.png",
                bbox_inches='tight', pad_inches=1.2)

In [ ]:
names = ['Azimuth', 'Elevation', 'Rotator', 'Dome', 'M2']
counts = np.zeros([5])
for lastInPosition in lastInPositions:
    for i, name in enumerate(names):
        if lastInPosition == name:
            counts[i] += 1
fig, ax = plt.subplots()
colors = plt.cm.Pastel1.colors
wedge_properties = {'linewidth': 1, 'edgecolor': 'black'}
ax.pie(counts, labels=names, autopct='%1.1f%%', colors=colors, 
       wedgeprops = wedge_properties, startangle=90) # Add labels and percentages
ax.axis('equal')  # Ensures the pie chart is drawn as a circle
ax.set_title("Last component in position", fontsize=18, y=1.05)
print(len(lastInPositions), counts.sum())
fig.savefig(f"/home/c/cslage/u/MTMount/between_image_timing/LastInPosition_{dayObs}.png",
                bbox_inches='tight', pad_inches=1.2)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(maxDeltas, interval_times[1])
ax.set_ylim(0, 300)
ax.set_xlabel("Slew distance (max of Alt, Az) (degrees)")
ax.set_ylabel("Time from slewStart to allInposition (seconds)")
ax.set_title(f"Slew time vs slew distance - {dayObs}", fontsize=18, y=1.05)
fig.savefig(f"/home/c/cslage/u/MTMount/between_image_timing/Slew_Times_{dayObs}.png",
                bbox_inches='tight', pad_inches=1.2)